In [1]:
pip install shutil

ERROR: Could not find a version that satisfies the requirement shutil (from versions: none)
ERROR: No matching distribution found for shutil
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

# Liệt kê các thư mục trong /kaggle/input/
input_dirs = os.listdir('/kaggle/input/')
print("Các thư mục trong /kaggle/input/:", input_dirs)

# Giả sử tên thư mục là 'my-dataset', liệt kê các tệp trong đó
dataset_path = '/kaggle/input/checkpoint-1050'
files = os.listdir(dataset_path)
print(f"Các tệp trong {dataset_path}:", files)

Các thư mục trong /kaggle/input/: ['checkpoint-1050']
Các tệp trong /kaggle/input/checkpoint-1050: ['checkpoint-1050']


In [18]:
rm -rf /kaggle/working/trainer_output

In [3]:
import shutil
# Đường dẫn đến thư mục nguồn và đích
src = '/kaggle/input/checkpoint-1050/checkpoint-1050'
dst = '/kaggle/working/trainer_output/checkpoint-1050'

# Sao chép toàn bộ thư mục
shutil.copytree(src, dst)


FileExistsError: [Errno 17] File exists: '/kaggle/working/trainer_output/checkpoint-1050'

In [4]:
pip install unsloth vllm

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()
accelerator.free_memory()

In [5]:
from unsloth import FastModel
import torch


fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

MODEL = "unsloth/gemma-3-12b-it-unsloth-bnb-4bit"
MODEL_NAME = "gemma-3-12b-it-unsloth-bnb-4bit"
DATASET = "nguyenphuthien/vi-alpaca-data"
CHAT_TEMPLATE = "gemma-3"
OUTPUT_DIR = "./trainer_output"
SAVE_LORA_DIR = "./lora-adapter"
SAVE_DIR = "./finetuned-merged"
max_seq_length = 2048
load_in_4bit = True
load_in_8bit = False
full_finetuning = False
    





🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-05-08 02:04:35.075827: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746669875.102430     282 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746669875.110838     282 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-08 02:04:43 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-08 02:04:43 [__init__.py:239] Automatically detected platform cuda.


In [6]:
from unsloth import FastModel

model, tokenizer = FastModel.from_pretrained(
    model_name = MODEL,
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = load_in_4bit,  # 4 bit quantization to reduce memory
    load_in_8bit = load_in_8bit, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = full_finetuning, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)


==((====))==  Unsloth 2025.4.7: Fast Gemma3 patching. Transformers: 4.51.1. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [7]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.language_model.model` require gradients


In [8]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = CHAT_TEMPLATE,
)

In [10]:
from huggingface_hub import login
from datasets import load_dataset
from kaggle_secrets import UserSecretsClient

hf_token = UserSecretsClient().get_secret("HF_TOKEN")
login(token=hf_token)

dataset = load_dataset(DATASET, split = "train")

from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)
def format_example(example):
    # Combine instruction and input (if input exists and is not empty)
    user_content = example['instruction']
    if example['input'] and example['input'].strip():
        user_content += f"\n{example['input']}"
        
    # Create the conversations structure
    conversations = [
        {'content': user_content, 'role': 'user'},
        {'content': example['output'], 'role': 'assistant'}
    ]
        
    return {'conversations': conversations}
dataset = dataset.map(format_example)

def formatting_prompts_func(examples):
   convos = examples["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False).removeprefix('<bos>') for convo in convos]
   return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)




README.md:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

alpaca.jsonl:   0%|          | 0.00/221M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/292081 [00:00<?, ? examples/s]

Map:   0%|          | 0/292081 [00:00<?, ? examples/s]

Map:   0%|          | 0/292081 [00:00<?, ? examples/s]

In [14]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        output_dir = OUTPUT_DIR,
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = 10,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
        dataset_num_proc=2,
        save_strategy="steps",
        save_steps= 20,
        save_total_limit=1
    ),
)


TypeError: SFTConfig.__init__() got an unexpected keyword argument 'weights_only'

In [15]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)





Map (num_proc=4):   0%|          | 0/292081 [00:00<?, ? examples/s]

AttributeError: 'list' object has no attribute 'tolist'

In [ ]:
trainer_stats = trainer.train(
    resume_from_checkpoint=False,
)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 292,081 | Num Epochs = 1 | Total steps = 18,255
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 32,735,232/12,000,000,000 (0.27% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.113600
20,1.064900
30,1.076600
40,1.044900
50,0.981500
60,1.001800
70,1.083100
80,1.047600


In [ ]:
model.save_pretrained(SAVE_LORA_DIR)  # Local saving
tokenizer.save_pretrained(SAVE_LORA_DIR)

if False:
    model.save_pretrained_merged(
        SAVE_DIR,
        tokenizer
    )

In [ ]:
!zip -r /kaggle/working/trainer_output.zip /kaggle/working/trainer_output

In [ ]:
# Define the save directory and repository name
SAVE_DIR = "gemma-3-finetune"
REPO_NAME = "gemma-3-12b-it-unsloth-bnb-4bit-finetune"

# Authenticate with Hugging Face Hub
api = HfApi()

# Create a new repository or update an existing one
api.create_repo(REPO_NAME, private=False)

# Upload the model and tokenizer to the Hugging Face Hub
upload_folder(
    folder_path=SAVE_DIR,
    repo_id="tieubaoca/" + REPO_NAME,
    commit_message="Upload compressed model",
)


print(f"Model and tokenizer successfully pushed to Hugging Face Hub: {REPO_NAME}")